# Setup

In [590]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [591]:
from contextlib import redirect_stdout
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [592]:
estimation_method = 'hardware-capex-opex'
compute_threshold_method = 'backward_window_percentile'  # top_n, window_percentile, backward_window_percentile, residual_from_trend
compute_threshold = 85  # e.g. 10 to select top 10; 75 to select top 25%
variant = 'original'
exclude_models_containing = []

estimation_method_lookup = {
    'cloud': estimate_cloud_costs,
    'amortized': estimate_amortized_hardware_costs,
    'hardware-acquisition': estimate_hardware_acquisition_cost,
    'hardware-capex-opex': estimate_hardware_capex_opex,
}
cost_estimation_function = estimation_method_lookup[estimation_method]

results_dir = f'results/{estimation_method}-{compute_threshold_method}={compute_threshold}-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [593]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_threshold_method=compute_threshold_method, compute_threshold=compute_threshold,
)

In [594]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(68, 5510, 142)

# Cost estimation

In [595]:
with open(f'{results_dir}/cost_estimation.out', 'w') as f:
    with redirect_stdout(f):
        cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

In [596]:
if estimation_method == 'hardware-capex-opex':
    frontier_pcd_df_copy = frontier_pcd_df.copy()
    with open(f'{results_dir}/component_cost_estimation.out', 'w') as f:
        with redirect_stdout(f):
            component_cost_df = cost_estimation_function(frontier_pcd_df_copy, hardware_df, price_df, separate_components=True)

In [597]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Organization categorization (from Organization),Possibly over 1e23 FLOP,Cost
82,Gemini Ultra,"Multimodal,Language,Vision","Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,633.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN,Industry,NaN,6.126676e+07
90,Qwen-72B,Language,"Chat,Code generation","Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Ka...",SOTA improvement,"SOTA on several Chinese benchmarks, with highe...",Open access (restricted use),https://huggingface.co/Qwen/Qwen-72B,NaN,NaN,...,NaN,NaN,NaN,NaN,Unreleased,Unreleased,up to 100m active users:\r\nhttps://github.com...,Industry,NaN,NaN
98,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,Hosted access (no API),https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,checked,NaN,NaN,NaN,NaN,NaN,"via Pi, no API",Industry,NaN,1.171711e+07
123,Grok-1,Language,"Language modelling,Chat",NaN,SOTA improvement,"""On these benchmarks, Grok-1 displayed strong ...",Open source,"https://x.ai/model-card/, https://x.ai/blog/gr...",NaN,Announcing Grok,...,checked,2,7.0,NaN,Unreleased,Unreleased,apache 2.0,Industry,checked,NaN
128,Yi-34B,Language,Chat,"Alex Young, Bei Chen, Chao Li, Chengen Huang, ...",Significant use,2nd most popular model on HuggingFace: https:/...,Open access (restricted use),https://arxiv.org/abs/2403.04652,NaN,Yi: Open Foundation Models by 01.AI,...,NaN,NaN,NaN,NaN,NaN,NaN,apply for commercial:\r\nhttps://github.com/01...,Industry,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,NASv3 (CIFAR-10),Vision,NaN,"Barret Zoph, Quoc V. Le",Highly cited,NaN,NaN,https://arxiv.org/abs/1611.01578,4781.0,Neural Architecture Search with Reinforcement ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
1159,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,Hosted access (no API),https://arxiv.org/abs/1609.08144,6196.0,Google's Neural Machine Translation System: Br...,...,NaN,NaN,NaN,414720.0,NaN,NaN,presumably deployed via Google translate,Industry,NaN,1.146647e+05
1201,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14887.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
1207,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2749.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,NaN,NaN,1920.0,NaN,NaN,NaN,Industry,NaN,1.725745e+02


In [598]:
cost_df['Cost'].notna().sum()

47

Exclusion

In [599]:
for kw in exclude_models_containing:
    cost_df = cost_df[cost_df['System'].str.contains(kw) == False]
list(cost_df['System'])

['Gemini Ultra',
 'Qwen-72B',
 'Inflection-2',
 'Grok-1',
 'Yi-34B',
 'ChatGLM3',
 'Falcon-180B',
 'Llama 2-70B',
 'Claude 2',
 'xTrimoPGLM -100B',
 'PaLM 2',
 'GPT-4',
 'LLaMA-65B',
 'GPT-3.5 (text-davinci-003)',
 'BLOOM-176B',
 'U-PaLM (540B)',
 'Flan-PaLM 540B',
 'BlenderBot 3',
 'Minerva (540B)',
 'Parti',
 'OPT-175B',
 'PaLM (540B)',
 'Chinchilla',
 'ST-MoE',
 'LaMDA',
 'GPT-NeoX-20B',
 'AlphaCode',
 'ERNIE 3.0 Titan',
 'GLaM',
 'Gopher (280B)',
 'Yuan 1.0',
 'Megatron-Turing NLG 530B',
 'HyperCLOVA',
 'FLAN 137B',
 'GOAT',
 'CoAtNet',
 'ByT5-XXL',
 'ProtT5-XXL',
 'Meta Pseudo Labels',
 'Switch',
 'DALL-E',
 'mT5-XXL',
 'GPT-3 175B (davinci)',
 'Turing-NLG',
 'Meena',
 'OpenAI Five',
 'OpenAI Five Rerun',
 'AlphaStar',
 'T5-11B',
 'Megatron-BERT',
 'Megatron-LM (8.3B)',
 'RoBERTa Large',
 'XLNet',
 'MnasNet-A3',
 'MnasNet-A1 + SSDLite',
 'BERT-Large-CAS (PTB+WT2+WT103)',
 'GPT-2 (1.5B)',
 'BigGAN-deep 512x512',
 'FTW',
 'ResNeXt-101 32x48d',
 'AlphaZero',
 'AlphaGo Zero',
 'AlphaG

Use the below to check data availability for specific systems

In [600]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [601]:
cost_df['Cost'].dropna()

82      6.126676e+07
98      1.171711e+07
184     9.362940e+06
223     9.938486e+05
235     1.646685e+06
278     9.684345e+06
319     3.739479e+07
335     5.180867e+05
379     4.222085e+06
390     8.022369e+05
405     5.941722e+06
407     5.917571e+06
430     1.303369e+06
453     6.361336e+06
459     6.876996e+05
496     6.531276e+05
506     5.906844e+06
528     3.770427e+05
530     1.618202e+05
557     1.077455e+06
561     1.024991e+06
602     3.250046e+06
609     5.145633e+05
614     3.781393e+05
629     1.419045e+05
643     3.157644e+03
651     1.547803e+05
659     1.436324e+05
697     9.048363e+04
713     2.343297e+05
716     1.037081e+05
785     1.820445e+06
820     4.461402e+04
825     3.480160e+05
852     2.091735e+05
856     1.255628e+05
870     6.613550e+05
871     9.014862e+04
896     6.960159e+04
920     1.572243e+04
921     1.572243e+04
982     8.578783e+03
1055    6.833213e+05
1071    1.036716e+06
1131    7.916623e+05
1159    1.146647e+05
1207    1.725745e+02
Name: Cost, d

In [602]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [603]:
cost_df['Cost (inflation-adjusted)'].dropna()

82      6.126676e+07
98      1.174444e+07
184     9.420876e+06
223     9.995083e+05
235     1.656062e+06
278     9.783886e+06
319     3.779583e+07
335     5.246990e+05
379     4.306936e+06
390     8.183594e+05
405     6.061445e+06
407     6.036806e+06
430     1.329358e+06
453     6.481348e+06
459     7.006736e+05
496     6.634560e+05
506     6.004609e+06
528     3.897914e+05
530     1.672917e+05
557     1.111858e+06
561     1.057719e+06
602     3.352803e+06
609     5.317725e+05
614     3.907859e+05
629     1.464025e+05
643     3.265432e+03
651     1.600638e+05
659     1.485353e+05
697     9.349111e+04
713     2.431737e+05
716     1.076222e+05
785     1.910809e+06
820     4.678733e+04
825     3.656134e+05
852     2.228981e+05
856     1.338014e+05
870     7.053802e+05
871     9.614964e+04
896     7.490604e+04
920     1.698208e+04
921     1.698208e+04
982     9.240941e+03
1055    7.387380e+05
1071    1.120792e+06
1131    8.645025e+05
1159    1.252149e+05
1207    1.919755e+02
Name: Cost (i

In [604]:
assert cost_df['Cost (inflation-adjusted)'].notna().sum() == cost_df['Cost'].notna().sum()

# Regression

In [605]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [606]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.420
Model:                            OLS   Adj. R-squared:                  0.407
Method:                 Least Squares   F-statistic:                     32.60
Date:                Fri, 17 May 2024   Prob (F-statistic):           8.41e-07
Time:                        16:08:27   Log-Likelihood:                -55.225
No. Observations:                  47   AIC:                             114.4
Df Residuals:                      45   BIC:                             118.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -681.6477    120.383     -5.662      0.000    -924.112    -439.183
x1             0.3401      0.060      5.710      0.000       0.220       0.460
==============================================================================
Omnibus:                        0.941   Durbin-Watson:                   1.346
Prob(Omnibus):                  0.625   Jarque-Bera (JB):                0.324
Skew:                          -0.136   Prob(JB):                        0.850
Kurtosis:                       3.303   Cond. No.                     2.08e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.08e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [607]:
with open(f'{results_dir}/regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(reg_results)
print_growth_rates(reg_results)

N=47.0
R^2=0.42
0.34 OOMs/year (95% CI: 0.22, 0.46)
2.2x/year (95% CI: 1.7x, 2.9x)
doubling time of 11 months (95% CI: 8, 16)


In [608]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [609]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,3.582071,0.386971,2.802671,4.361471,1.790820,5.373322,2015.00000,2015-01-01
1,3.616421,0.381240,2.848564,4.384278,1.830162,5.402680,2015.10101,2015-02-06
2,3.650771,0.375518,2.894439,4.407102,1.869436,5.432106,2015.20202,2015-03-15
3,3.685121,0.369805,2.940296,4.429946,1.908641,5.461600,2015.30303,2015-04-21
4,3.719471,0.364101,2.986133,4.452809,1.947777,5.491165,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,6.845315,0.233872,6.374273,7.316358,5.165138,8.525493,2024.59596,2024-08-06
96,6.879665,0.239103,6.398087,7.361244,5.196504,8.562827,2024.69697,2024-09-12
97,6.914015,0.244370,6.421828,7.406203,5.227788,8.600243,2024.79798,2024-10-19
98,6.948365,0.249671,6.445501,7.451229,5.258990,8.637740,2024.89899,2024-11-25


# Export data

In [610]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
82,Gemini Ultra,"Multimodal,Language,Vision","Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,6.126676e+07,6.126676e+07
90,Qwen-72B,Language,"Chat,Code generation",Open access (restricted use),NaN,2023-11-30,Alibaba,7.200000e+10,1.300000e+24,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Inflection-2,Language,Language modelling,Hosted access (no API),Inflection-2: The Next Step Up,2023-11-22,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,1.171711e+07,1.174444e+07
123,Grok-1,Language,"Language modelling,Chat",Open source,Announcing Grok,2023-11-04,xAI,3.140000e+11,2.900000e+24,NaN,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,Yi-34B,Language,Chat,Open access (restricted use),Yi: Open Foundation Models by 01.AI,2023-11-02,01.AI,3.400000e+10,6.100000e+23,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,NASv3 (CIFAR-10),Vision,NaN,NaN,Neural Architecture Search with Reinforcement ...,2016-11-05,Google Brain,3.740000e+07,2.200000e+21,50000.0,...,NaN,United States of America,NaN,NaN,800.0,NaN,NaN,NaN,NaN,NaN
1159,GNMT,Language,Translation,Hosted access (no API),Google's Neural Machine Translation System: Br...,2016-09-26,Google,2.780000e+08,6.900000e+21,360000000.0,...,NVIDIA Tesla K80,United States of America,NaN,NaN,96.0,NaN,NaN,NaN,1.146647e+05,1.252149e+05
1201,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27,DeepMind,NaN,1.900000e+21,29400000.0,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1207,DeepSpeech2 (English),Speech,Speech recognition,NaN,Deep Speech 2: End-to-End Speech Recognition i...,2015-12-08,Baidu Research - Silicon Valley AI Lab,3.800000e+07,2.600000e+19,163339200.0,...,NVIDIA GTX Titan X,United States of America,NaN,NaN,16.0,0.45,NaN,NaN,1.725745e+02,1.919755e+02


In [611]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)

# Plots

In [612]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [613]:
label_systems = ['GNMT', 'AlphaGo Master', 'AlphaGo Zero', 'AlphaZero', 'DALL-E', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra', 'Inflection-2']

tpu_mask = cost_df['Training hardware'].str.contains('TPU', na=False)
tpu_cost_df = cost_df.loc[tpu_mask]
gpu_cost_df = cost_df.loc[~tpu_mask]

fig = px.scatter(
    gpu_cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    log_y=True,
)
fig.add_scatter(
    x=tpu_cost_df['Publication date'],
    y=tpu_cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    marker_symbol='circle-open' if estimation_method != 'cloud' else 'circle',
    name='Using equivalent price of TPU' if estimation_method != 'cloud' else '',
    showlegend=estimation_method != 'cloud',
)
fig.add_scatter(
    x=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)
fig.add_scatter(
    x=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])
if estimation_method == 'hardware-acquisition':
    fig.update_yaxes(range=[4, 10])
else:
    fig.update_yaxes(range=[1, 9])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Cost components

In [614]:
cost_component_names = [
    'AI accelerator chip cost',
    'Other server components cost',
    'Cluster-level interconnect cost',
    'Energy cost',
]

In [615]:
for key in cost_component_names:
    component_cost_df[f"{key} (%)"] = component_cost_df[key] / component_cost_df['Cost'] * 100
component_cost_df['AI accelerator chip cost (%)']

82      50.488092
90            NaN
98      49.753696
123           NaN
128           NaN
          ...    
1150          NaN
1159    47.246922
1201          NaN
1207    35.825588
1212          NaN
Name: AI accelerator chip cost (%), Length: 68, dtype: float64

In [616]:
cost_component_pc_names = [name + ' (%)' for name in cost_component_names]
filtered_component_cost_df = component_cost_df.dropna(subset=cost_component_pc_names).sort_values(by='Publication date')

In [617]:
# Stacked bar chart of cost components, using component_cost_df
fig = px.bar(
    filtered_component_cost_df,
    x='System',
    y=cost_component_pc_names,
    barmode='stack',
)

# axis labels
fig.update_xaxes(title_text='ML model')
fig.update_yaxes(title_text='% of hardware CapEx + OpEx')
fig.update_layout(
    legend=dict(
        title_text='Cost component',
        x=0.60,
        y=0.05,
    )
)
# limits 0 to 100
fig.update_yaxes(range=[0, 100])

fig.update_yaxes(tickvals=list(range(0, 101, 10)))

# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_component_percentage')

fig.show()

In [618]:
fig = px.bar(
    filtered_component_cost_df,
    x='System',
    y='Energy cost (%)',
    barmode='stack',
    # labels='Cost %',
    # text='Energy cost %',
)
# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Energy cost (% of Hardware CapEx + OpEx)')
# fig.update_layout(
#     legend=dict(
#         title_text='Cost component',
#         x=0.75,
#         y=0.05,
#     )
# )
# limits 0 to 100
fig.update_yaxes(range=[0, 30])
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_percentage')

fig.show()

In [619]:
fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y='Energy cost',
    text='System',
)
# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Energy cost')
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_cost')

fig.show()

In [620]:
from energy import energy_price

# Stacked bar chart of cost components, using component_cost_df
filtered_component_cost_df.loc[:, 'Energy (kWh)'] = [
    row['Energy cost'] / energy_price(row['Publication date'].year) 
    for _, row in filtered_component_cost_df.iterrows()
]
fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y='Energy (kWh)',
    text='System',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_kwh')

fig.show()

In [621]:
filtered_component_cost_df = filtered_component_cost_df.dropna(subset=['Training hardware'])
filtered_component_cost_df.loc[:, 'Power capacity for 3-month training run (kW)'] = [
    row['Energy cost'] \
    / energy_price(row['Publication date'].year) \
    / (HOURS_PER_YEAR/4) \
    / server_TDP_fraction(row['Training hardware']) \
    for _, row in filtered_component_cost_df.iterrows()
]

fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y='Power capacity for 3-month training run (kW)',
    text='System',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'power_capacity_kw')

fig.show()

In [622]:
filtered_component_cost_df['Publication date (float)'] = datetime_to_float_year(
    pd.to_datetime(filtered_component_cost_df['Publication date'])
)

In [623]:
energy_reg_results = fit_ols_regression(
    filtered_component_cost_df,
    ['Publication date (float)'],
    'Power capacity for 3-month training run (kW)',
    logy=True
)
energy_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     47.30
Date:                Fri, 17 May 2024   Prob (F-statistic):           1.55e-08
Time:                        16:08:44   Log-Likelihood:                -48.466
No. Observations:                  47   AIC:                             100.9
Df Residuals:                      45   BIC:                             104.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -714.5301    104.258     -6.853      0.000    -924.517    -504.543
x1             0.3547      0.052      6.877      0.000       0.251       0.459
==============================================================================
Omnibus:                        1.044   Durbin-Watson:                   1.485
Prob(Omnibus):                  0.593   Jarque-Bera (JB):                0.397
Skew:                          -0.159   Prob(JB):                        0.820
Kurtosis:                       3.319   Cond. No.                     2.08e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.08e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""